In [142]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import json
from autofaiss import build_index


['..', '..', '..', '..', '..', '..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmpf_d004qf', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/u

In [143]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv")
clean_df.head()

,Unnamed: 0,test_caption,test_imgs
0,0,feiyangg/LP Paragraph Style Electric Guitar Ti...,O1CN01r2RqZk1lJ4Sk2ZYaN_!!2200643934797.jpg
1,1,Summer Lace White Shorts Pants Female Thin Ant...,O1CN01H0o2pi2HAtGC9rVWa_!!2206736519111.jpg
2,2,High-Quality Export 8MM with Adhesive Cork Cor...,O1CN01qeHW0Z1D3mRPfmNvM_!!161-0-lubanu.jpg
3,3,LED Ceiling Living Room Bedroom Corridor Corri...,O1CN01phQoOe25ljjN1lpLO_!!3082327567.jpg
4,4,Disposable Cotton Cotton Female Thin Water Fac...,O1CN01Dzpgpr2FxJsUzqAdE_!!2963188946.jpg


In [144]:
image_embed_path = "./image_embeds"
text_embed_path = "./text_embeds"
image_embed_files = os.listdir(image_embed_path)
text_embed_files = os.listdir(text_embed_path)
image_embed_files.remove("val_set_embed")
text_embed_files.remove("val_set_embed")

In [145]:
text_embed_files

['txt_emded-large-large_v2_e_5.pkl',
 'txt_emded-base-base_1st_e_5.pkl',
 'txt_emded-base-base_3rd_e_5.pkl',
 'txt_emded-base-base_2nd_e_5.pkl',
 'txt_emded-base-base_all_e_7.pkl',
 'txt_emded-base-base_5th_e_5.pkl',
 'txt_emded-large-large_all_e_9.pkl',
 'txt_emded-large-large_v3_e_1.pkl',
 'txt_emded-base-base_4th_e_5.pkl',
 'txt_emded-large-large_v1_e_5.pkl']

In [146]:
all_image_embed ={}
all_text_embed = {}
for i,t in tqdm(zip(image_embed_files,text_embed_files)):
    with open(f"{image_embed_path}/{i}","rb") as img_file:
        img_embed = pickle.load(img_file)
        all_image_embed[img_embed["model_name"]] = img_embed["embed"]
    with open(f"{text_embed_path}/{t}","rb") as txt_file:
        txt_embed = pickle.load(txt_file)
        all_text_embed[txt_embed["model_name"]] = txt_embed["embed"]


10it [00:01,  7.14it/s]


In [147]:
all_image_embed.keys()

dict_keys(['large_v2_e_5', 'base_5th_e_5', 'base_all_e_7', 'base_2nd_e_5', 'base_1st_e_5', 'large_v1_e_5', 'base_3rd_e_5', 'base_4th_e_5', 'large_v3_e_1', 'large_all_e_9'])

In [148]:
all_text_embed.keys()

dict_keys(['large_v2_e_5', 'base_1st_e_5', 'base_3rd_e_5', 'base_2nd_e_5', 'base_all_e_7', 'base_5th_e_5', 'large_all_e_9', 'large_v3_e_1', 'base_4th_e_5', 'large_v1_e_5'])

In [149]:
pair_to_test = "large_all_e_9"

In [150]:
embed_to_ensamble = ["base_5th_e_5","large_v2_e_5","large_v3_e_1"]

# embed_to_ensamble = list(all_text_embed.keys())

In [151]:
# all_text_embed["base_all_e_7"] = all_text_embed["base_all_e_7"][0:20000]

In [152]:
# def get_ensamble(all_image_embed,all_text_embed,embeds_to_ensamble,ensamble_strategy):
#     ensamsables = {}
#     img_embed_list = []
#     text_embed_list = []
#     for model_name in embeds_to_ensamble:
#         img_embed_list.append(all_image_embed[model_name])
#         text_embed_list.append(all_text_embed[model_name])
#     if len(embeds_to_ensamble) > 1:
#         if ensamble_strategy =="concat":
#             ensamsables["image"] = torch.concat(img_embed_list,dim=1)
#             ensamsables["text"] = torch.concat(text_embed_list,dim=1)
#         elif ensamble_strategy =="avg":
#             ensamsables["image"] = torch.mean(torch.stack(img_embed_list))
#             ensamsables["text"] = torch.mean(torch.concat(text_embed_list))
#         return ensamsables
#     else:
#             ensamsables["image"] = img_embed_list
#             ensamsables["text"] = text_embed_list
#             return ensamsables


        
# def get_ensamble(all_image_embed,all_text_embed,embeds_to_ensamble,ensamble_strategy):
#     ensamsables = {}
#     img_embed_list = []
#     text_embed_list = []
#     for model_name in embeds_to_ensamble:
#         print(model_name)
#         img_embed_list.append(all_image_embed[model_name])
#         text_embed_list.append(all_text_embed[model_name])
#     if len(embeds_to_ensamble) > 1:
#         if ensamble_strategy =="concat":
#             ensamsables["image"] = torch.concat(img_embed_list,dim=1)
#             ensamsables["text"] = torch.concat(text_embed_list,dim=1)
#         elif ensamble_strategy =="avg":
#             ensamsables["image"] = torch.mean(torch.stack(img_embed_list),0)
#             ensamsables["text"] = torch.mean(torch.stack(text_embed_list),0)
#         else:
#             ensamsables["image"] = img_embed_list
#             ensamsables["text"] = text_embed_list
#         return ensamsables
#     else:
#             ensamsables["image"] = img_embed_list[0]
#             ensamsables["text"] = text_embed_list[0]
#             return ensamsables



def get_ensamble(all_image_embed,all_text_embed,embeds_to_ensamble,ensamble_strategy,all_weights):
    ensamsables = {}
    img_embed_list = []
    text_embed_list = []
    all_weights = all_weights
    for model_name ,weights in zip(embeds_to_ensamble,range(len(all_weights))):
        img_embed_list.append(all_image_embed[model_name]*all_weights[weights])
        text_embed_list.append(all_text_embed[model_name]*all_weights[weights])
    if len(embeds_to_ensamble) > 1:
        if ensamble_strategy =="concat":
            ensamsables["image"] = torch.concat(img_embed_list,dim=1)
            ensamsables["text"] = torch.concat(text_embed_list,dim=1)
        elif ensamble_strategy =="avg":
            ensamsables["image"] = torch.mean(torch.stack(img_embed_list),0)
            ensamsables["text"] = torch.mean(torch.stack(text_embed_list),0)
        else:
            ensamsables["image"] = img_embed_list
            ensamsables["text"] = text_embed_list
        return ensamsables
    else:
            ensamsables["image"] = img_embed_list[0]
            ensamsables["text"] = text_embed_list[0]
            return ensamsables

In [153]:
# ensamsables = get_ensamble(all_image_embed,all_text_embed,embed_to_ensamble,"concat")

ensamsables = get_ensamble(all_image_embed,all_text_embed,embed_to_ensamble,"concat",[1.5,2,0.8])

In [154]:

# def get_preds(ensamsables,score=False,df=None):
#     image_embed = ensamsables["image"]
#     text_embed = ensamsables["text"]
#     faiss_index = faiss.IndexFlatIP(image_embed.shape[1])
#     faiss_index.add(image_embed.numpy().astype(np.float32))
#     top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
#     top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
#     if score:
#         top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
#         top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
#         top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
#         print(f"top_5 accuracy:{top5_acc}")
#         top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
#         avg_acc = (top5_acc+top10_acc)/2
#         print(f"top_5 accuracy:{top10_acc}")
#         print(f"avg accuracy:{avg_acc}")
#         return top5_acc,top10_acc,avg_acc
#     else:
#         return top5_k_y_pred,top10_k_y_pred,top5_k_e,top10_k_e


def get_preds(ensamsables,score=False,df=None,use_faiss=True,auto_faiss=False,max_ensamble=False,sharpen_weight = 1):
    image_embed = ensamsables["image"]
    text_embed = ensamsables["text"]
    if use_faiss:
        faiss_index = faiss.IndexFlatIP(image_embed.shape[1])
        faiss_index.add(image_embed.numpy().astype(np.float32))
        top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
        top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    elif auto_faiss:
        auto_faiss_index,index_infos  = build_index(image_embed.numpy().astype(np.float32), save_on_disk=False)
        top5_k_e,top5_k_y_pred = auto_faiss_index.search(text_embed.numpy().astype(np.float32),5)
        top10_k_e,top10_k_y_pred = auto_faiss_index.search(text_embed.numpy().astype(np.float32),10)

    elif max_ensamble:
        all_top10_cosine_sim  =[]
        all_top5_cosine_sim  =[]
        all_top10_pred_idx  =[]
        all_top5_pred_idx  =[]
        for text,image in zip(text_embed,image_embed):
            faiss_index = faiss.IndexFlatIP(image.shape[1])
            faiss_index.add(image.numpy().astype(np.float32))
            top5_cosine_sim,top5_pred_idx = faiss_index.search(text.numpy().astype(np.float32),5)
            top10_cosine_sim,top10_pred_idx = faiss_index.search(text.numpy().astype(np.float32),10)
            all_top10_cosine_sim.append(top10_cosine_sim)
            all_top5_cosine_sim.append(top5_cosine_sim) 
            all_top10_pred_idx.append(top10_pred_idx)
            all_top5_pred_idx.append(top5_pred_idx)
        all_top10_cosine_sim  = np.concatenate(all_top10_cosine_sim,axis=1)
        all_top5_cosine_sim  = np.concatenate(all_top5_cosine_sim,axis=1)
        all_top10_pred_idx  = np.concatenate(all_top10_pred_idx,axis=1)
        all_top5_pred_idx  = np.concatenate(all_top5_pred_idx,axis=1)
        top5_k_y_pred = []
        top10_k_y_pred = []
        for t5,t10,t5_i,t10_i in zip(all_top5_cosine_sim,all_top10_cosine_sim,all_top5_pred_idx,all_top10_pred_idx):
            top5_k_y_pred.append(t5_i[np.argpartition(t5,-5)[-5:]])
            top10_k_y_pred.append(t10_i[np.argpartition(t10,-10)[-10:]])
        print(top10_k_y_pred)

    else:
        all_cosine_sim  =[]
        all_weights = [0.3,0.5,0.2]
        for text,image,w_i in zip(text_embed,image_embed,range(len(all_weights))):
            cosine_sim = text @ image.T 
            all_cosine_sim.append(cosine_sim * all_weights[w_i])
            del cosine_sim
        # ensamble_cosine_sim = torch.mean(torch.stack(all_cosine_sim),dim=0) 
        # ensamble_cosine_sim = 1 / torch.mean(1. / (torch.stack(all_cosine_sim)) +0.0001,dim=0) 
        ensamble_cosine_sim = torch.prod(torch.stack(all_cosine_sim),dim=0)**1/3
        top5_k_e,top5_k_y_pred = torch.topk(ensamble_cosine_sim,5)
        top10_k_e,top10_k_y_pred = torch.topk(ensamble_cosine_sim,10)
    if score:
            top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
            top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
            top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
            top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
            avg_acc = (top5_acc+top10_acc)/2
            return top5_acc,top10_acc,avg_acc
    else:
        return top5_k_y_pred,top10_k_y_pred

In [155]:
# top5_pred_idx,top10_pred_idx,top5_k_e,top10_k_e = get_preds(ensamsables,score=False,df=clean_df)
top5_pred_idx,top10_pred_idx = get_preds(ensamsables,score=False,df=clean_df,use_faiss=True,auto_faiss=False)

In [156]:
def remap_pred_to_name(top5_pred_idx,top10_pred_idx,image_array):
    top5_pred = [list(image_array[p]) for p in top5_pred_idx]
    top10_pred =  [list(image_array[p]) for p in top10_pred_idx]
    return top5_pred,top10_pred

In [157]:
top5_pred,top10_pred =remap_pred_to_name(top5_pred_idx,top10_pred_idx,clean_df["test_imgs"].values)

In [158]:
def results_to_json(top5_pred,top10_pred,caption_array,result_dir,result_file_name):
    results = []
    for c,t5,t10 in tqdm(zip(caption_array,top5_pred,top10_pred)):
        result_dict = {}
        result_dict["caption"] = c
        result_dict["top5"] = t5
        result_dict["top10"] = t10
        results.append(result_dict)
    print("checking results format")
    print(results[0])
    with open(f"{result_dir}/{result_file_name}.json","w") as results_file:
        json.dump(results,results_file)
    with open(f"{result_dir}/{result_file_name}.json","r") as results_file:
        check_results = json.load(results_file)
    print("checking results format of saved file")
    print(check_results[-1])
    return check_results
    
        



In [159]:
saved_result = results_to_json(top5_pred,top10_pred,clean_df["test_caption"].values[0:20000],"./submissions","test_ensamble_top3_w_v2_8_6_22")

20000it [00:00, 753226.48it/s]


checking results format
{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized', 'top5': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg', 'O1CN011pW88rTaqfnNIi1_!!754885367.jpg', 'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png', 'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg', 'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg'], 'top10': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg', 'O1CN011pW88rTaqfnNIi1_!!754885367.jpg', 'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png', 'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg', 'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg', 'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg', 'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg', 'TB1C3GMkUD1gK0jSZFGL6Td3FXa', 'TB2lylSaQ.OyuJjSszhXXbZbVXa_!!20295181.jpg', 'O1CN01ha825u1ctPVv8XwDP_!!2211470413658-0-picasso.jpg']}
checking results format of saved file
{'caption': 'mrs harmay foods Handmade Cantonese Barbecued Pork Bag 390g/Bag', 'top5': ['O1CN01472J632BQ6ioi13iS_!!2680068332.

In [160]:
len(saved_result)

20000

In [161]:
saved_result[:1]

[{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized',
  'top5': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg',
   'O1CN011pW88rTaqfnNIi1_!!754885367.jpg',
   'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png',
   'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg',
   'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg'],
  'top10': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg',
   'O1CN011pW88rTaqfnNIi1_!!754885367.jpg',
   'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png',
   'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg',
   'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg',
   'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg',
   'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg',
   'TB1C3GMkUD1gK0jSZFGL6Td3FXa',
   'TB2lylSaQ.OyuJjSszhXXbZbVXa_!!20295181.jpg',
   'O1CN01ha825u1ctPVv8XwDP_!!2211470413658-0-picasso.jpg']}]

In [162]:
top5_pred_idx 

array([[27694, 47178, 35939, 40609, 42193],
       [23383, 19321,  5591, 44309, 16976],
       [31777, 10273, 21649, 34609, 16947],
       ...,
       [24440,  7107, 18232, 14769, 43062],
       [12274, 39630,  4278, 18451, 20602],
       [35529, 43082,  6883, 18811, 35923]])